# Rules of choosing a good password
In this notebook, we will go through the rules in NIST Special Publication 800-63B which details what checks a verifier (what the NIST calls a second party responsible for storing and verifying passwords) should perform to make sure users don't pick bad passwords. We will go through the passwords of users from a fictional company and use R to flag the users with bad passwords. 

The NIST guidelines are:

         1. Verifiers SHALL require subscriber-chosen memorized secrets to be at least 8 characters in length
         2. Verifiers SHALL compare the prospective secrets against a list that contains values known to be commonly-used, expected, or compromised.
           a. Passwords obtained from previous breach corpuses.
           b. Dictionary words.
           c. Repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).
           d. Context-specific words, such as the name of the service, the username, and derivatives thereof.

# Importing the tidyverse library


In [74]:
library(tidyverse)

# Loading the data and examining it


In [60]:
# Loading in datasets/users.csv 
users <-read_csv("datasets/users.csv")


# Counting how many users we've got
count(users)

# Taking a look at the 12 first users

head(users,12)

Parsed with column specification:
cols(
  id = col_double(),
  user_name = col_character(),
  password = col_character()
)


n
<int>
982


id,user_name,password
<dbl>,<chr>,<chr>
1,vance.jennings,joobheco
2,consuelo.eaton,0869347314
3,mitchel.perkins,fabypotter
4,odessa.vaughan,aharney88
5,araceli.wilder,acecdn3000
6,shawn.harrington,5278049
7,evelyn.gay,master
8,noreen.hale,murphy
9,gladys.ward,lwsves2


# Checking the length of passwords

According to NIST guidelines, passwords should not be shorter than 8 characters in length. We will now flag users with short passwords.


In [62]:
# Calculating the lengths of users' passwords
users <- users %>% 
         mutate(length = nchar(password))


# Flagging the users with too short passwords

users <- users %>% 
         mutate(too_short = if_else(length <8, TRUE, FALSE))

# Counting the number of users with too short passwords

users %>% count(too_short == TRUE)

# Taking a look at the 12 first rows

head(users, 12)

too_short == TRUE,n
<lgl>,<int>
FALSE,606
TRUE,376


id,user_name,password,length,too_short
<dbl>,<chr>,<chr>,<int>,<lgl>
1,vance.jennings,joobheco,8,FALSE
2,consuelo.eaton,0869347314,10,FALSE
3,mitchel.perkins,fabypotter,10,FALSE
4,odessa.vaughan,aharney88,9,FALSE
5,araceli.wilder,acecdn3000,10,FALSE
6,shawn.harrington,5278049,7,TRUE
7,evelyn.gay,master,6,TRUE
8,noreen.hale,murphy,6,TRUE
9,gladys.ward,lwsves2,7,TRUE


# Commonly used passwords
Already this simple rule flagged a couple of offenders among the first 12 users. Next we will check passwords obtained from previous breach corpuses, that is, websites where hackers have leaked all the users' passwords. As many websites don't follow the NIST guidelines and encrypt passwords there now exist large lists of the most popular passwords. Let's start by loading in the 10,000 most common passwords which taken from here.

In [75]:
# Reading in the top 10000 passwords
common_passwords <- read_tsv("datasets/10_million_password_list_top_10000.txt", col_names = FALSE)

# Taking a look at the top 100
head(common_passwords, 100)


Parsed with column specification:
cols(
  X1 = col_character()
)


X1
<chr>
123456
password
12345678
qwerty
123456789
12345
1234
111111
1234567


# Passwords should not be common passwords

Let's flag all the passwords in our user database that are among the top 10,000 used passwords.

In [68]:
# Flagging the users with passwords that are common passwords

users <- users %>%
         mutate(common_password = password %in% common_passwords$X1)
# Counting the number of users using common passwords

users %>%count(common_password)

# Taking a look at the 12 first rows
head(users,12)

common_password,n
<lgl>,<int>
FALSE,853
TRUE,129


id,user_name,password,length,too_short,common_password
<dbl>,<chr>,<chr>,<int>,<lgl>,<lgl>
1,vance.jennings,joobheco,8,FALSE,FALSE
2,consuelo.eaton,0869347314,10,FALSE,FALSE
3,mitchel.perkins,fabypotter,10,FALSE,FALSE
4,odessa.vaughan,aharney88,9,FALSE,FALSE
5,araceli.wilder,acecdn3000,10,FALSE,FALSE
6,shawn.harrington,5278049,7,TRUE,FALSE
7,evelyn.gay,master,6,TRUE,TRUE
8,noreen.hale,murphy,6,TRUE,TRUE
9,gladys.ward,lwsves2,7,TRUE,FALSE


# Passwords should not be common words
It turns out many of our users use common passwords, and of the first 12 users there are already two. However, as most common passwords also tend to be short, they were already flagged as being too short. 
Let's check our users' passwords against the top 10,000 English words from Google's Trillion Word Corpus.

In [69]:
# Reading in a list of the 10000 most common words
words <- read_tsv("datasets/google-10000-english.txt", col_name = FALSE)


# Flagging the users with passwords that are common words
users<- users %>% 
         mutate(common_word = password %in% words$X1)

                      

# Counting the number of users using common words as passwords
users %>% count(common_word)

# Taking a look at the 12 first rows
head(users,12)

Parsed with column specification:
cols(
  X1 = col_character()
)


common_word,n
<lgl>,<int>
FALSE,846
TRUE,136


id,user_name,password,length,too_short,common_password,common_word
<dbl>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>
1,vance.jennings,joobheco,8,FALSE,FALSE,FALSE
2,consuelo.eaton,0869347314,10,FALSE,FALSE,FALSE
3,mitchel.perkins,fabypotter,10,FALSE,FALSE,FALSE
4,odessa.vaughan,aharney88,9,FALSE,FALSE,FALSE
5,araceli.wilder,acecdn3000,10,FALSE,FALSE,FALSE
6,shawn.harrington,5278049,7,TRUE,FALSE,FALSE
7,evelyn.gay,master,6,TRUE,TRUE,TRUE
8,noreen.hale,murphy,6,TRUE,TRUE,TRUE
9,gladys.ward,lwsves2,7,TRUE,FALSE,FALSE


# Passwords should not be name
Our users' usernames consist of their first names and last names separated by a dot. Let's flag passwords that are the same as either a user's first or last name

In [70]:
# Extracting first and last names into their own columns
users <- users %>%
         mutate(user_name2split = user_name)
users <- users %>% 
       separate(user_name2split, c("first_name","second_name"))


# Flagging the users with passwords that matches their names

users <- users %>% 
        mutate(uses_name = str_detect(password, first_name)  | str_detect(password, second_name))


# Counting the number of users using names as passwords
users %>% count(uses_name)

# Taking a look at the 12 first rows
head(users,12)

uses_name,n
<lgl>,<int>
FALSE,932
TRUE,50


id,user_name,password,length,too_short,common_password,common_word,first_name,second_name,uses_name
<dbl>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<lgl>
1,vance.jennings,joobheco,8,FALSE,FALSE,FALSE,vance,jennings,FALSE
2,consuelo.eaton,0869347314,10,FALSE,FALSE,FALSE,consuelo,eaton,FALSE
3,mitchel.perkins,fabypotter,10,FALSE,FALSE,FALSE,mitchel,perkins,FALSE
4,odessa.vaughan,aharney88,9,FALSE,FALSE,FALSE,odessa,vaughan,FALSE
5,araceli.wilder,acecdn3000,10,FALSE,FALSE,FALSE,araceli,wilder,FALSE
6,shawn.harrington,5278049,7,TRUE,FALSE,FALSE,shawn,harrington,FALSE
7,evelyn.gay,master,6,TRUE,TRUE,TRUE,evelyn,gay,FALSE
8,noreen.hale,murphy,6,TRUE,TRUE,TRUE,noreen,hale,FALSE
9,gladys.ward,lwsves2,7,TRUE,FALSE,FALSE,gladys,ward,FALSE


# Passwords should not be repititive
To check for repetitiveness can be arbitrarily complex, but here we're only going to do something simple. We're going to flag all passwords that contain 4 or more repeated characters.

In [72]:
# Splitting the passwords into vectors of single characters
split_passwords <- strsplit(as.character(users$password), "")

# Picking out the max number of repeat characters for each password

users$max_repeats <- sapply(split_passwords, function(split_password) {
    x <- split_password
    repeats<- character(length(x))
    for (i in 1:length(x)){ 
    repeats[i]<-sum(str_count(x, fixed(x[i])))
    }
   max_repeats <- max(repeats)
   return(max_repeats)
})

# Flagging the passwords with >= 4 repeats
users<- users %>% 
        mutate(too_many_repeats = if_else(max_repeats >=4, TRUE, FALSE))

# Taking a look at the users with too many repeats
users %>% filter(too_many_repeats)

id,user_name,password,length,too_short,common_password,common_word,first_name,second_name,uses_name,max_repeats,too_many_repeats
<dbl>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<lgl>
39,gus.padilla,wwewwf1,7,TRUE,FALSE,FALSE,gus,padilla,FALSE,4,TRUE
89,ira.cortez,mmommy22,8,FALSE,FALSE,FALSE,ira,cortez,FALSE,4,TRUE
147,patti.dixon,555555,6,TRUE,TRUE,FALSE,patti,dixon,FALSE,6,TRUE
181,pierre.hooper,10101971,8,FALSE,FALSE,FALSE,pierre,hooper,FALSE,4,TRUE
193,trinidad.austin,012227257,9,FALSE,FALSE,FALSE,trinidad,austin,FALSE,4,TRUE
200,tara.mckenzie,eddiestoocool,13,FALSE,FALSE,FALSE,tara,mckenzie,FALSE,4,TRUE
286,lee.roman,alamaage678,11,FALSE,FALSE,FALSE,lee,roman,FALSE,4,TRUE
329,elma.campos,ddpdtd3nii,10,FALSE,FALSE,FALSE,elma,campos,FALSE,4,TRUE
365,emery.ruiz,888278,6,TRUE,FALSE,FALSE,emery,ruiz,FALSE,4,TRUE


# Flagging all bad passwords
Now we have implemented all the basic tests for bad passwords suggested by NIST Special Publication 800-63B! What's left is just to flag all bad passwords and maybe to send these users an e-mail that strongly suggests they change their password.

In [73]:
# Flagging all passwords that are bad
#users$bad_password <- ....
users <- users %>%
          mutate( bad_password = if_else(
            too_short == TRUE | common_password == TRUE | common_word == TRUE | uses_name == TRUE | too_many_repeats == TRUE, "yes", "no"
         ))
            
# Counting the number of bad passwords
users %>% count(bad_password)

# Looking at the first 100 bad passwords
bad_pwd_users <- users %>% filter(bad_password == "yes")
head(bad_pwd_users$password, 100)

bad_password,n
<chr>,<int>
no,538
yes,444


[1] "5278049"       "master"        "murphy"        "lwsves2"      
  [5] "hubbard"       "310356"        "oZ4k0QE"       "chelsea"      
  [9] "zvc1939"       "nickgd"        "cocacola"      "woodard"      
 [13] "AJ9Da"         "ewokzs"        "YyGjz8E"       "reid"         
 [17] "jOYZBs8"       "wwewwf1"       "225377"        "NdZ7E6"       
 [21] "CQB3Z"         "diffo"         "123456789"     "y8uM7D6"      
 [25] "mikeloo"       "golden"        "apr2502"       "jfasii"       
 [29] "sanaang"       "jecarn"        "dakami"        "thx1138"      
 [33] "midnight"      "7xFHcdy"       "dorian"        "2458098"      
 [37] "mmommy22"      "hurley"        "UYHUGG"        "wishes"       
 [41] "jake145"       "Oli#2b"        "monkey"        "shy!"         
 [45] "chifito"       "123ASH"        "juliette"      "kokjoon"      
 [49] "crystal"       "scanner"       "andrew"        "caleb"        
 [53] "dreker"        "h3ZFd"         "replaced"      "buster"       
 [57] "nu"            "ware"          "46ofus"        "rr55367"      
 [61] "fastman"       "biodiversity"  "555555"        "lara"         
 [65] "9786435"       "roland"        "gokuass"       "ov0moh"       
 [69] "stefan"        "9812684"       "TIRO02"        "dixie"        
 [73] "gtapbt"        "123654"        "1036900"       "flavor"       
 [77] "FGPb9gX"       "vHO2y"         "10101971"      "precise"      
 [81] "190664"        "4f3aRNG"       "austin"        "jtc123"       
 [85] "eLvtn"         "122412"        "8309113"       "012227257"    
 [89] "dillard"       "william"       "NCAA50"        "eddiestoocool"
 [93] "4101516"       "leyvi"         "ancan"         "blake"        
 [97] "tdutza"        "framework"     "golfer"        "winter"

# Summary
In this notebook, we've implemented the password checks recommended by the NIST Special Publication 800-63B. It's certainly possible to better implement these checks, for example, by using a longer list of common passwords. Also note that the NIST checks in no way guarantee that a chosen password is good, just that it's not obviously bad.